# Projet : Extraction et Recommandation de films et séries avec rdflib et SPARQL

## Objectif
L’objectif de ce mini-projet est l’extraction des données de la base de données DBPedia en utilisant le langage de requêtes SPARQL afin d'alimenter la base de données de séries que vous avez créé au premier TP, l’analyse de ces données et la recommandation de séries et de films.

# Description

Vous allez créer un système de recommandation de séries/films basé sur des données RDF qui stockent des informations sur les films, les utilisateurs et leurs préférences cinématographiques.

Les étapes du projet sont comme suit :

## 1.  Extraction des données

a)  Vous utiliserez Rdflib pour accéder aux données et vous effectuerez des requêtes SPARQL pour les extraire. Vous êtes libres dans le choix et la taille des données que vous allez extraire. Le jeu de données doit néanmoins être représentatif pour pouvoir fournir des recommandations précises. 
    
   - Les données à extraire : 
       - Films : Chaque film a un titre, un réalisateur, une année de sortie, un genre (e.g. : action, comédie, science-fiction), un résumé, une liste d'acteurs principaux, une durée, une évaluation du film, etc.
       - Réalisateur : Chaque réalisateur a un nom, une biographie et une liste de films qu'il a réalisés.
       - Acteurs : Chaque acteur a un nom, une biographie et une liste de films dans lesquels il a joué.
       - Genres :  Chaque genre a un nom et une description.
       - Utilisateurs : chaque utilisateur a un identifiant et des préférences cinématographiques (acteurs préférés, genres préférés, etc).
       - Évaluations : Elle est décrite par l'identifiant de l'utilisateur qui a donné l'évaluation, identifiant du film évalué la note attribuée au film, Commentaire ou avis sur le film.

   - Liens entre les entités : 

        - Les films sont associés à leurs acteurs, réalisateurs et genres.
        - Les utilisateurs sont associés aux films qu'ils ont évalués.
        - Les utilisateurs peuvent être liés entre eux en fonction de leurs préférences cinématographiques similaires.

   b)  Transformer les données en triplets RDF :  Vous allez transformer ces résultats en triplets RDF avant de les ajouter à votre graphe RDF existant. 
  
  c) Ajouter les données au graphe existant : Utilisez la méthode g.add() de votre graphe RDF, que vous avez créé au premier TP, pour ajouter les triplets RDF représentant les données DBpedia que vous avez transformées.

titre : Le Dernier Fiacre
titre : Cabaret (film)
titre : Baril de poudre
titre : Cabaret Paradis
titre : Cabeza de Vaca (film)
titre : Têtes coupées
titre : Cabin Boy
titre : Cabin Fever: Patient Zero
titre : Cabin Fever
titre : Cabin Fever (film, 2016)
titre : Cabin Fever: Spring Fever
titre : Un petit coin aux cieux (film, 1943)
titre : Cabiria
titre : Cabo Blanco (film)
titre : Chasse à la drogue
titre : Caché
titre : Cactus (film, 1986)
titre : Fleur de cactus (film, 1969)
titre : Cadavres (film)
titre : Le Golf en folie
titre : Le Golf en folie 2
titre : Cadence (film)
titre : Cadences obstinées
titre : Cadets en vacances
titre : Cadet Kelly
titre : Les Cadets
titre : Cadillac Man
titre : Cadillac Records
titre : César doit mourir
titre : César et Cléopâtre (film)
titre : Cafard (film, 2015)
titre : Cafe Funiculi Funicula
titre : Café Métropole (film)
titre : Femme du monde
titre : Butterfly Café
titre : Café Derby
titre : Café express
titre : Café Flesh
titre : Café Lumière
titre

In [30]:
from SPARQLWrapper import SPARQLWrapper, JSON

# URL du point d'accès SparQL de la DBpedia
dbpedia_sparql_endpoint = "http://dbpedia.org/sparql"

# Création d'une instance de SPARQLWrapper
sparql = SPARQLWrapper(dbpedia_sparql_endpoint)

# Requête SPARQL mise à jour
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?film ?title ?director ?releaseYear ?genre ?summary ?duration ?rating (GROUP_CONCAT(?actor; SEPARATOR=", ") AS ?actorsList)
WHERE {
    ?film rdf:type dbo:Film .                 # Les entités de type Film
    ?film rdfs:label ?title .                # Titre du film
    ?film dbo:director ?director .           # Réalisateur
    ?film dbo:releaseDate ?releaseYear .     # Année de sortie
    ?film dbo:genre ?genre .                 # Genre
    ?film dbo:abstract ?summary .            # Résumé
    ?film dbo:starring ?actor .              # Acteurs (URI des acteurs)
    ?film dbo:runtime ?duration .            # Durée
    OPTIONAL { ?film dbo:rating ?rating }    # Évaluation (optionnelle)
    FILTER (lang(?title) = "en")             # Filtrer les titres en anglais
    FILTER (lang(?summary) = "en")           # Filtrer les résumés en anglais
}
GROUP BY ?film ?title ?director ?releaseYear ?genre ?summary ?duration ?rating
LIMIT 100
"""

# Format de résultat en JSON
sparql.setReturnFormat(JSON)

# Exécution de la requête
sparql.setQuery(query)
results = sparql.query().convert()

# Affichage des résultats
for result in results["results"]["bindings"]:
    film = result.get("film", {}).get("value", "N/A")
    title = result.get("title", {}).get("value", "N/A")
    director = result.get("director", {}).get("value", "N/A")
    releaseYear = result.get("releaseYear", {}).get("value", "N/A")
    genre = result.get("genre", {}).get("value", "N/A")
    summary = result.get("summary", {}).get("value", "N/A")
    actorsList = result.get("actorsList", {}).get("value", "N/A")  # Liste des URI des acteurs
    duration = result.get("duration", {}).get("value", "N/A")
    rating = result.get("rating", {}).get("value", "N/A")

    print(f"Film: {film}")
    print(f"  Title: {title}")
    print(f"  Director: {director}")
    print(f"  Release Year: {releaseYear}")
    print(f"  Genre: {genre}")
    print(f"  Summary: {summary}")
    print(f"  Actors: {actorsList}")
    print(f"  Duration: {duration}")
    print(f"  Rating: {rating}\n")


Film: http://dbpedia.org/resource/Camp_Wilder
  Title: Camp Wilder
  Director: http://dbpedia.org/resource/Arlene_Sanford
  Release Year: 1992-09-18
  Genre: http://dbpedia.org/resource/Sitcom
  Summary: Camp Wilder is an American television sitcom which aired on ABC from September 18, 1992 until February 26, 1993. The premise centered on a young woman who opens up her home to the friends of her younger siblings, who sought it as judgment-free "hangout", and who regularly went to her for advice. The series was created by Matthew Carlson, and produced by a.k.a. Productions in association with Capital Cities Entertainment. The show aired as a part of ABC's TGIF lineup, but was cancelled after 19 episodes due to low ratings. A twentieth episode was produced, but was never aired in the United States. Camp Wilder was also shown in the UK, Spain and Germany.
  Actors (URIs): http://dbpedia.org/resource/Mary_Page_Keller, http://dbpedia.org/resource/Meghann_Haldeman, http://dbpedia.org/resourc

Réalisateur

In [31]:
# Requête SPARQL
query = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dbr: <http://dbpedia.org/resource/>

SELECT ?director ?name ?bio (GROUP_CONCAT(?film; SEPARATOR=", ") AS ?films)
WHERE {
    ?director a dbo:Person .                 # L'entité est une personne
    ?director dbo:occupation dbr:Film_director . # La personne est un réalisateur
    ?director foaf:name ?name .             # Nom du réalisateur
    ?director dbo:abstract ?bio .           # Biographie
    ?film dbo:director ?director .          # Film réalisé par cette personne
    FILTER (lang(?bio) = "en")              # Filtrer les biographies en anglais
}
GROUP BY ?director ?name ?bio
LIMIT 50
"""

# Format de résultat en JSON
sparql.setReturnFormat(JSON)

# Exécution de la requête
sparql.setQuery(query)
results = sparql.query().convert()

# Affichage des résultats
for result in results["results"]["bindings"]:
    director = result.get("director", {}).get("value", "N/A")
    name = result.get("name", {}).get("value", "N/A")
    bio = result.get("bio", {}).get("value", "N/A")
    films = result.get("films", {}).get("value", "N/A")  # Liste des films sous forme d'URI

    print(f"Director URI: {director}")
    print(f"  Name: {name}")
    print(f"  Bio: {bio}")
    print(f"  Films (URIs): {films}\n")


Director URI: http://dbpedia.org/resource/Cameron_Crowe
  Name: Cameron Crowe
  Bio: Cameron Bruce Crowe (born July 13, 1957) is an American journalist, author, writer, producer, director, actor, lyricist, and playwright. Before moving into the film industry, Crowe was a contributing editor at Rolling Stone magazine, for which he still frequently writes. Crowe's debut screenwriting effort, Fast Times at Ridgemont High (1982), grew out of a book he wrote while posing for one year undercover as a student at Clairemont High School in San Diego. Later, he wrote and directed another high school film, Say Anything... (1989), followed by Singles (1992), a story of twentysomethings that was woven together by a soundtrack centering on Seattle's burgeoning grunge music scene. Crowe landed his biggest hit with Jerry Maguire (1996). After this, he was given a green-light to go ahead with a pet project, the autobiographical film Almost Famous (2000). Centering on a teenage music journalist on tour 

Acteurs

In [32]:
# Requête SPARQL
query = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?actor ?name ?bio (GROUP_CONCAT(?film; SEPARATOR=", ") AS ?films)
WHERE {
    ?actor a dbo:Actor .                    # L'entité est un acteur
    ?actor foaf:name ?name .                # Nom de l'acteur
    ?actor dbo:abstract ?bio .              # Biographie
    ?film dbo:starring ?actor .             # Films dans lesquels l'acteur a joué
    FILTER (lang(?bio) = "en")              # Filtrer les biographies en anglais
}
GROUP BY ?actor ?name ?bio
LIMIT 50
"""

# Format de résultat en JSON
sparql.setReturnFormat(JSON)

# Exécution de la requête
sparql.setQuery(query)
results = sparql.query().convert()

# Affichage des résultats
for result in results["results"]["bindings"]:
    actor = result.get("actor", {}).get("value", "N/A")
    name = result.get("name", {}).get("value", "N/A")
    bio = result.get("bio", {}).get("value", "N/A")
    films = result.get("films", {}).get("value", "N/A")  # Liste des films sous forme d'URI

    print(f"Actor URI: {actor}")
    print(f"  Name: {name}")
    print(f"  Bio: {bio}")
    print(f"  Films: {films}\n")


Actor URI: http://dbpedia.org/resource/Camilla_Sparv
  Name: Camilla Sparv
  Bio: Camilla Sparv (born 3 June 1943) is a Swedish actress.
  Films (URIs): http://dbpedia.org/resource/The_Trouble_with_Angels_(film), http://dbpedia.org/resource/Mackenna's_Gold, http://dbpedia.org/resource/Downhill_Racer, http://dbpedia.org/resource/Nobody_Runs_Forever, http://dbpedia.org/resource/The_Greek_Tycoon, http://dbpedia.org/resource/Assignment_K, http://dbpedia.org/resource/Survival_Zone

Actor URI: http://dbpedia.org/resource/Candida_Royalle
  Name: Candida Royalle
  Bio: Candida Royalle (born Candice Marion Vadala; October 15, 1950 – September 7, 2015) was an American producer and director of couples-oriented pornography, pornographic actress, sex educator, and sex-positive feminist. She was a member of the XRCO and the AVN Halls of Fame.
  Films (URIs): http://dbpedia.org/resource/Hot_&_Saucy_Pizza_Girls, http://dbpedia.org/resource/The_Tale_of_Tiffany_Lust

Actor URI: http://dbpedia.org/resour

Genres

In [33]:
# Requête SPARQL
query = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?genre ?name ?description
WHERE {
    ?genre a dbo:Genre .                     # L'entité est un genre
    ?genre foaf:name ?name .                 # Nom du genre
    ?genre dbo:abstract ?description .       # Description du genre
    FILTER (lang(?description) = "en")       # Filtrer les descriptions en anglais
}
LIMIT 20
"""

# Format de résultat en JSON
sparql.setReturnFormat(JSON)

# Exécution de la requête
sparql.setQuery(query)
results = sparql.query().convert()

# Affichage des résultats
for result in results["results"]["bindings"]:
    genre = result.get("genre", {}).get("value", "N/A")
    name = result.get("name", {}).get("value", "N/A")
    description = result.get("description", {}).get("value", "N/A")

    print(f"Genre URI: {genre}")
    print(f"  Name: {name}")
    print(f"  Description: {description}\n")


Genre URI: http://dbpedia.org/resource/Cadence_rampa
  Name: Cadence rampa
  Description: Cadence rampa (Haitian Creole: kadans ranpa, [kadãs ɣãpa]), or simply kadans, is a dance music and modern méringue popularized in the Caribbean by the virtuoso Haitian sax player Webert Sicot in the early 1960s. Cadence rampa was one of the sources of cadence-lypso.Cadence and compas are two names for the same Haitian modern meringue.

Genre URI: http://dbpedia.org/resource/Car_song
  Name: Car song
  Description: A car song is a song with lyrics or musical themes pertaining to car travel. Though the earliest forms appeared in the 1900s, car songs emerged in full during the 1950s as part of rock and roll and car culture, but achieved their peak popularity in the West Coast of the United States during the 1960s with the emergence of hot rod rock as an outgrowth of the surf music scene. Though this popularity declined by the late 1960s, cars remain a frequently used subject matter in pop music into 

Utilisateur

In [34]:
# Requête SPARQL pour simuler des préférences utilisateurs
query = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX ex: <http://example.org/>

SELECT ?user ?userId ?preferredActor ?actorName ?preferredGenre ?genreName
WHERE {
    # Simulation d'utilisateurs fictifs
    ?user a ex:User .                        # L'entité utilisateur
    ?user ex:userId ?userId .                # Identifiant de l'utilisateur
    ?user ex:preferredActor ?preferredActor .# Acteur préféré
    ?preferredActor foaf:name ?actorName .   # Nom de l'acteur préféré
    ?user ex:preferredGenre ?preferredGenre .# Genre préféré
    ?preferredGenre foaf:name ?genreName .   # Nom du genre préféré

    # Exemples d'entités d'acteurs et de genres dans DBpedia
    ?preferredActor a dbo:Actor .            # Acteurs DBpedia
    ?preferredGenre a dbo:Genre .            # Genres DBpedia
}
LIMIT 20
"""

# Format de résultat en JSON
sparql.setReturnFormat(JSON)

# Exécution de la requête
sparql.setQuery(query)
results = sparql.query().convert()

# Affichage des résultats
for result in results["results"]["bindings"]:
    user = result.get("user", {}).get("value", "N/A")
    userId = result.get("userId", {}).get("value", "N/A")
    preferredActor = result.get("preferredActor", {}).get("value", "N/A")
    actorName = result.get("actorName", {}).get("value", "N/A")
    preferredGenre = result.get("preferredGenre", {}).get("value", "N/A")
    genreName = result.get("genreName", {}).get("value", "N/A")

    print(f"User URI: {user}")
    print(f"  User ID: {userId}")
    print(f"  Preferred Actor: {actorName} ({preferredActor})")
    print(f"  Preferred Genre: {genreName} ({preferredGenre})\n")


Evaluations

In [35]:
# Requête SPARQL pour récupérer les évaluations
query = """
PREFIX ex: <http://example.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?user ?userId ?film ?filmTitle ?rating ?comment
WHERE {
    ?evaluation a ex:Evaluation .            # Une entité de type Evaluation
    ?evaluation ex:user ?user .              # Utilisateur qui a donné l'évaluation
    ?user ex:userId ?userId .                # Identifiant de l'utilisateur
    ?evaluation ex:film ?film .              # Film évalué
    ?film foaf:name ?filmTitle .             # Titre du film
    ?evaluation ex:rating ?rating .          # Note attribuée au film
    ?evaluation ex:comment ?comment .        # Commentaire de l'utilisateur
}
LIMIT 20
"""

# Format de résultat en JSON
sparql.setReturnFormat(JSON)

# Exécution de la requête
sparql.setQuery(query)
results = sparql.query().convert()

# Affichage des résultats
for result in results["results"]["bindings"]:
    user = result.get("user", {}).get("value", "N/A")
    userId = result.get("userId", {}).get("value", "N/A")
    film = result.get("film", {}).get("value", "N/A")
    filmTitle = result.get("filmTitle", {}).get("value", "N/A")
    rating = result.get("rating", {}).get("value", "N/A")
    comment = result.get("comment", {}).get("value", "N/A")

    print(f"User: {user} (ID: {userId})")
    print(f"  Film: {filmTitle} ({film})")
    print(f"  Rating: {rating}")
    print(f"  Comment: {comment}\n")


## 2. Prétraitement des données   

Nettoyez et traitez les données extraites pour supprimer les doublons, gérer les valeurs manquantes et normaliser.

## 3. Analyse exploratoire des données 

- Créer des graphiques permettant de visualiser la distribution des films et séries dans votre base de données.
- Créer un graphique pour montrer les films et les séries les mieux notés
- Créer un nuage de points pour représenter la relation entre les caractéristiques

## 4. Système de Recommandation 

a) Utiliser SPARQL pour interroger le graphe RDF afin de créer un système de recommandation de films et/ou séries. Vous pouvez envisager différentes approches de recommandation, telles que la recommandation collaborative (en fonction des évaluations d'utilisateurs similaires) ou la recommandation basée sur le contenu (en fonction des genres, des acteurs, etc.) ou la recommandation basée sur les connaissances

b) Utiliser une IA (ChatGPT ou tout autre) pour répondre à cette question

## 5. Calcul des Recommandations 

a) Utiliser SPARQL pour générer des requêtes de recommandation en fonction des préférences de l'utilisateur. Vous pouvez également utiliser des algorithmes d'apprentissage automatique pour améliorer les recommandations.

b) Utiliser une IA pour répondre à cette question

## 6. Évaluation  

a) Évaluez la qualité de vos recommandations en utilisant des mesures telles que  MAP@k (Mean Average Precision at k) ou NDCG@k (Normalized Discounted Cumulative Gain at k)

b) Comparer les recommandations que vous avez obtenues par rapport à celles générées par l'IA que vous aurez utilisée

## 7. Rapport 

Vous allez rédiger un rapport de 5 pages max décrivant la modélisation RDF, les requêtes SPARQL, l'algorithme de recommandation et les résultats de l'évaluation. Vous spécifierez votre utilisation de l'IA dans le cadre de ce projet et ce que vous en pensez